<a href="https://colab.research.google.com/github/abuelnasr0/Semantic-Search-project/blob/populate_vectorDB/Notebooks/vector_database/populate_vdb_with_captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install grpcio==1.49.1
!pip install pymilvus==2.2.8

In [ ]:
!pip install sentence_transformers
!pip install torch

In [41]:
uri = ""
user = ""
password = ""

In [42]:
from pymilvus import connections, Collection

connections.connect(
  alias="default", 
  uri=uri, 
  secure=True,
  user=user, 
  password=password
)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import json
with open('/content/drive/MyDrive/Graduation_project/Data/Youtube_captions/Captions_with_stride/PLUl4u3cNGP619EG1wp0kT-7rDE_Az5TNd.json') as json_file:
    playlist_captions = json.load(json_file)
playlist_videos = playlist_captions['videos']

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model_name = 'sentence-transformers/msmarco-bert-base-dot-v5'
model = SentenceTransformer(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [43]:
id = 0

ids = []
videos_ids = []
embeddings = []
start_times = []
texts = []

for videos in playlist_videos:
  video_id = videos['video_id']
  for caption in videos['video_captions'] :
    ids.append(id)
    id+=1
    embeddings.append(model.encode(caption["text"]).tolist())
    videos_ids.append(video_id)
    start_times.append(caption["start"])
    texts.append(caption["text"])

data = [ids, videos_ids, embeddings, start_times, texts]


In [45]:
videos_collection = Collection("youtube_videos")     
videos_collection.load()
videos_collection.insert(data)
videos_collection.flush()
print(str(videos_collection.num_entities))

1414


In [104]:
# Search test 
query = "machine learning"
query_embedding = model.encode(query)
query_E = query_embedding.tolist()
top_k = 15 

search_params = {"metric_type": "IP"}



results = videos_collection.search(
    data = [query_E],
    anns_field = "embedding",
    param = search_params,
    output_fields=['text', 'start_time', 'video_id'],
    limit = top_k
)

# print(results[0].distances)


print(results[0][0].entity.get("text"))

Now it's been a problem in AI for a very long time.It's from a gentleman named Art Samuel. 1959 is the quote in which he says, his definition of machine learning is the field of study that gives computers the ability to learn without being explicitly programmed. And I think many people would argue, he wrote the first such program. It learned from experience. 
